In [ ]:
! pip install fuzzywuzzy

In [3]:
import pandas as pd
import time
import mlflow
import calendar
import re
from datetime import timedelta
from fuzzywuzzy import fuzz, process
from transformers import pipeline
from huggingface_hub import hf_hub_download
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

# Set MLflow remote URI
mlflow.set_tracking_uri("http://3.145.180.106:5000")
mlflow.set_experiment("Cincinnati Crime Chatbot")

# Load dataset
def load_crime_data():
    csv_path = hf_hub_download(
        repo_id="mlsystemsg1/cincinnati-crime-data",
        repo_type="dataset",
        filename="calls_for_service_latest.csv"
    )
    df = pd.read_csv(csv_path, low_memory=False)
    df.columns = [col.lower() for col in df.columns]
    df.dropna(subset=['cpd_neighborhood'], inplace=True)
    df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], errors='coerce')
    return df

# Incident type grouping
OFFENSE_GROUPS = {
    "robbery": [
        "ROBBERY PERSONAL (JO)(W)", "ROBBERY PERSONAL (IP)", "ROBBERY BUSINESS (NIP)"
    ],
    "assault": [
        "ASSAULT (IP)(W)", "ASSAULT (JO)(W)(E)", "SEX ASSAULT ADULT (S)",
        "SEX ASSAULT CHILD (IP)", "SEX ASSAULT CHILD (S)(W)(E)", "RAPER"
    ],
    "disturbance": [
        "DISTURBANCE VERB (IP)(W)", "DISTURBANCE VERB (S)", "DISTURBANCE PHYS (JO)",
        "DISTURBANCE VERB (S)(W)", "DISTURBANCE PHYS (S)(E)",
        "FAMILY DIST PHYS (JO)(E)", "FAMILY DIST PHYS (S)", "FAMILY DIST UNKN (JO)"
    ],
    "theft": [
        "VEHICLE THEFT (JO)(W)", "U-AUTO THEFT/RECOVERY RPT",
        "THEFT (IP)(W)", "OCR THEFT ATTEMPT (NIP)"
    ],
    "drug": [
        "DRUG USE/POSSESS (IP)(W)(E)", "DRUG SALE (IP)(W)(E)", "ADV - DRUG SALE (NIP)"
    ]
}

# Filtering function
def get_relevant_rows(question, df):
    q = question.lower()
    filtered = df.copy()

    print(f"[DEBUG] Question: {q}")

    # Neighborhood filter
    neighborhoods = df['cpd_neighborhood'].dropna().unique()
    matched_hood = next((hood for hood in neighborhoods if hood.lower() in q), None)
    if matched_hood:
        filtered = filtered[filtered['cpd_neighborhood'].str.lower() == matched_hood.lower()]
        print(f"[DEBUG] Neighborhood match: {matched_hood}")
    else:
        print("[DEBUG] No neighborhood match.")

    # Offense group filter
    offense_matched_group = None
    for group, values in OFFENSE_GROUPS.items():
        if group in q:
            offense_matched_group = group
            filtered = filtered[filtered['incident_type_id'].isin(values)]
            print(f"[DEBUG] Offense group match: {group}")
            break

    # Fuzzy fallback
    if not offense_matched_group:
        unique_offenses = df['incident_type_id'].dropna().astype(str).unique()
        match, score = process.extractOne(q, unique_offenses, scorer=fuzz.partial_ratio)
        if score >= 85:
            filtered = filtered[filtered['incident_type_id'].str.lower() == match.lower()]
            print(f"[DEBUG] Fuzzy offense match: {match} (score: {score})")
        else:
            print("[DEBUG] No offense or fuzzy match found.")

    # Date match
    now = pd.Timestamp.now()
    if match := re.search(r"(20\d{2})[-/](\d{1,2})[-/](\d{1,2})", q):
        y, m, d = map(int, match.groups())
        filtered = filtered[filtered['create_time_incident'].dt.date == pd.Timestamp(y, m, d).date()]
        print(f"[DEBUG] Exact date matched: {y}-{m:02}-{d:02}")

    if "last week" in q or "past week" in q:
        filtered = filtered[filtered['create_time_incident'] >= now - timedelta(days=7)]
    elif "last month" in q or "past month" in q:
        filtered = filtered[filtered['create_time_incident'] >= now - timedelta(days=30)]
    elif "yesterday" in q:
        filtered = filtered[filtered['create_time_incident'].dt.date == (now - timedelta(days=1)).date()]
    elif "today" in q:
        filtered = filtered[filtered['create_time_incident'].dt.date == now.date()]
    elif match := re.search(r"(past|last)\s+(\d+)\s+day", q):
        days = int(match.group(2))
        filtered = filtered[filtered['create_time_incident'] >= now - timedelta(days=days)]

    months = {month.lower(): i for i, month in enumerate(calendar.month_name) if month}
    for name, num in months.items():
        if name in q:
            filtered = filtered[filtered['create_time_incident'].dt.month == num]
            print(f"[DEBUG] Month filter applied: {name}")
            break

    if year_match := re.search(r"(20\d{2})", q):
        filtered = filtered[filtered['create_time_incident'].dt.year == int(year_match.group(1))]

    filtered = filtered.dropna(subset=['create_time_incident'])

    if any(x in q for x in ["latest", "most recent", "last offense", "last incident", "recent"]):
        filtered = filtered.sort_values(by='create_time_incident', ascending=False)

    print(f"[DEBUG] Final row count: {len(filtered)}")
    return filtered

# Summary generator
def generate_summary(question, df):
    if df.empty:
        print("[DEBUG] No records matched after filtering.")
        return "No matching records found. The dataset contains no relevant incidents."

    print(f"[DEBUG] {len(df)} records matched. Preparing summary...")
    if any(word in question.lower() for word in ["how many", "count", "number of"]):
        return f"There were {len(df)} incidents matching your query."

    examples = []
    for _, row in df.head(10).iterrows():
        date = row.get('create_time_incident', 'N/A')
        try:
            date_str = pd.to_datetime(date).date()
        except:
            date_str = date
        offense = row.get('incident_type_id', 'N/A')
        hood = row.get('cpd_neighborhood', 'N/A')
        incident = row.get('event_number', 'N/A')
        priority = row.get('priority', 'N/A')
        examples.append(f"On {date_str}, a {offense} (Priority {priority}) occurred in {hood} (Incident #{incident}).")
    return "\n".join(examples)

# LLM response
def answer_with_llm(question, data_rows, model, model_name="google/flan-t5-base", prompt_version="v5"):
    if data_rows.empty:
        return "Sorry, I couldn't find any data matching that question."
    context = generate_summary(question, data_rows)
    prompt = f"""
You are a helpful assistant analyzing Cincinnati crime data.

Here are some relevant data points:
{context}

Now answer this question based on the above:
{question}
    """.strip()
    start_time = time.time()
    with mlflow.start_run():
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("question", question)
        mlflow.log_param("num_rows_context", len(data_rows))
        mlflow.log_param("prompt_version", prompt_version)
        mlflow.log_text(prompt, "prompt.txt")
        result = model(prompt, max_new_tokens=150)[0]['generated_text']
        answer = result.strip()
        latency = time.time() - start_time
        mlflow.log_text(answer, "answer.txt")
        mlflow.log_metric("response_length", len(answer))
        mlflow.log_metric("latency_sec", latency)
        mlflow.log_metric("match_count", len(data_rows))
        return answer

# One-turn interaction
def run_single_chatbot_turn(df, model, question):
    if question.lower() in ['exit', 'quit']:
        return "Goodbye!"
    rows = get_relevant_rows(question, df)
    print(f"[DEBUG] Filtered DataFrame contains {len(rows)} rows.")
    return answer_with_llm(question, rows, model)

# Widget UI
def ask_question_with_widget(df, model):
    input_box = widgets.Text(
        placeholder='Ask a question about crime in Cincinnati...',
        description='Question:',
        layout=widgets.Layout(width='100%')
    )
    output = widgets.Output()

    def on_submit(change):
        output.clear_output()
        question = change.value  # ✅ Correct way to get input value on submit
        with output:
            print("Bot:", run_single_chatbot_turn(df, model, question))

    input_box.on_submit(on_submit)
    display(input_box, output)

# Load model and data
print("Loading dataset and model...")
df = load_crime_data()
print("Columns in dataset:", df.columns.tolist())

llm_model = pipeline("text2text-generation", model="google/flan-t5-base")
print("\n✅ Chatbot is ready! Ask your question below:")
ask_question_with_widget(df, llm_model)

df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], errors='coerce')
print("\nMost recent incident date in dataset:")
print(df['create_time_incident'].max())
print("\nOldest incident date:")
print(df['create_time_incident'].min())
print("\nNumber of missing date entries:")
print(df['create_time_incident'].isna().sum())


Loading dataset and model...
Columns in dataset: ['create_time_incident', 'incident_type_desc', 'incident_type_id', 'disposition_text', 'event_number', 'sna_neighborhood', 'cpd_neighborhood', 'priority']


Device set to use cpu



✅ Chatbot is ready! Ask your question below:


Text(value='', description='Question:', layout=Layout(width='100%'), placeholder='Ask a question about crime i…

Output()


Most recent incident date in dataset:
2025-04-16 00:01:04

Oldest incident date:
2014-09-30 23:52:08

Number of missing date entries:
814
